In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/My Drive/AI/Stock prediction/'

/content/gdrive/My Drive/AI/Stock prediction


In [6]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-l8v2_95i
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-l8v2_95i
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0-cp36-none-any.whl size=80585 sha256=fe4e130dbec4350df6f1c80895ca97243df38c5d4a6566e53e68547d8ccaf567
  Stored in directory: /tmp/pip-ephem-wheel-cache-ovohysdy/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


In [0]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras import backend as kr
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling

In [10]:
Target=pd.read_csv("Videos/Random6.csv",header=None)
NumberofFrames=Target.shape[0]
TrainingData=np.zeros((NumberofFrames,288,432,3))
TrainingTarget=Target.values
Episodes=1
for Episode in range(Episodes):
  vidcap = cv2.VideoCapture("Videos/Random6.mp4")

  success,image = vidcap.read()
  
  count = 0
  Images=np.expand_dims(image,axis=0);
  while success:
    try:
      success,image = vidcap.read()
      #print('Read a new frame: ', success)
      image=np.expand_dims(image,axis=0)
      count += 1
      Images=np.concatenate([Images,image],axis=0)
    except:
      break
  #TrainingData=np.append(TrainingData,Images[:-1,:,:,:],axis=0)
  TrainingData=Images
print(TrainingData.shape)
print(TrainingTarget.shape)
TrainingData=(TrainingData-128.0)/256.0
TrainingTarget=TrainingTarget/500.0

(823, 288, 432, 3)
(823, 4)


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import os



train_data, Validation_Data, train_lbl, validation_lbl = train_test_split( TrainingData, TrainingTarget, test_size=1/10.0, random_state=0)

model = models.Sequential()
model.add(layers.Conv2D(3, (3, 3), activation='relu', input_shape=(288, 432, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='sigmoid'))
model.summary()
optimizer = tf.keras.optimizers.RMSprop(0.1)
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['mae'])
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
try:
  latest = tf.train.latest_checkpoint(checkpoint_dir)
  model.load_weights(latest)
  print('loaded')
except Exception:
  pass
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
history = model.fit(train_data, train_lbl, epochs=50, 
                    validation_data=(Validation_Data, validation_lbl),callbacks=[cp_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 286, 430, 3)       84        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 143, 215, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 141, 213, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 70, 106, 64)       0         
_________________________________________________________________
flatten (Flatten)            (None, 474880)            0         
_________________________________________________________________
dense (Dense)                (None, 4)                 1899524   
Total params: 1,901,400
Trainable params: 1,901,400
Non-trainable params: 0
______________________________________________

In [0]:
print((model.predict(train_data[21:22,:,:,:])*500).astype(int))
print((train_lbl[21,:]*500).astype(int))

[[339 281  48 209]]
[340 284  49 207]
